# Lecture 4.6 - Basics of cleaning messy text files 
## Part 2 - Grouping blocks of data and extracting information

In this lecture, we will go over a number of cases of messy data, and how to use Python to fix these problems.  This includes

1. Removing unwanted lines.
2. Parsing lines with regular expressions.
3. Working with data blocks spread across multiple lines.

## Reading in current progress

In [2]:
with open('911_Deaths_Grouped.csv') as f:
    content = f.read()
content[:500]

"Gordon M. Aamoth, Jr., 32, Sandler O'Neill + Partners, World Trade Center.\nEdelmiro Abad, 54, Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.\nMarie Rose Abad, 49, Keefe, Bruyette&Woods, Inc., World Trade Center.\nAndrew Anthony Abate, 37, Melville, N.Y., Cantor Fitzgerald, World Trade Center.\nVincent Paul Abate, 40, Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.\nLaurence Christopher Abel, 37, New York City, Cantor Fitzgerald, World Trade Center.\nAlona Abraham, 3"

In [3]:
grouped_lines = content.split('\n')
grouped_lines[:20]

["Gordon M. Aamoth, Jr., 32, Sandler O'Neill + Partners, World Trade Center.",
 'Edelmiro Abad, 54, Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.',
 'Marie Rose Abad, 49, Keefe, Bruyette&Woods, Inc., World Trade Center.',
 'Andrew Anthony Abate, 37, Melville, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Vincent Paul Abate, 40, Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Laurence Christopher Abel, 37, New York City, Cantor Fitzgerald, World Trade Center.',
 'Alona Abraham, 30, Ashdod, Israel, Passenger, United 175, World Trade Center.',
 'William F. Abrahamson, 55, Westchester County, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Richard Anthony Aceto, 42, Marsh&McLennan Companies, Inc., World Trade Center.',
 'Heinrich Bernhard Ackermann, 38, Aon Corporation, World Trade Center.',
 'Paul Acquaviva, 29, Glen Rock, N.J., Cantor Fitzgerald, World Trade Center.',
 'Christian Adams, 37, Passenger, United 93, Shanksville, Pa.',


## Preprocessing 

Below I have transfered over the preprocessing functions and applied them to the data.

In [4]:
# Imports
from composable import pipeable
from composable.strict import map

In [5]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))

In [6]:
(grouped_lines
>> map(add_missing_period)
>> map(fix_world_trade)
)

antor Fitzgerald, World Trade Center.',
 'Anthony J. Fallone, Jr., 39, New York City, Cantor Fitzgerald, World Trade Center.',
 'Dolores Brigitte Fanelli, 38, Farmingville, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Robert John Fangman, 33, Chelsea, Mass., Flight Crew, United 175, World Trade Center.',
 'John Joseph Fanning, 54, West Hempstead, N.Y., New York City Fire Department, World Trade Center.',
 'Kathleen Anne Faragher, 33, Risk Waters Group conference attendee from Janus Capital Group, World Trade Center.',
 'Thomas James Farino, 37, Bohemia, N.Y., New York City Fire Department, World Trade Center.',
 'Nancy C. Doloszycki Farley, 45, Jersey City, N.J., Reinsurance Solutions, World Trade Center.',
 'Paige Marie Farley-Hackel, 46, Newton, Mass., Passenger, United 11, World Trade Center.',
 'Elizabeth Ann Farmer, 62, Cantor Fitzgerald contractor, World Trade Center.',
 'Douglas Jon Farnum, 33, Brooklyn, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.'

In [7]:
# For convenience I will give these a name
prepped_lines = (grouped_lines 
                >> map(add_missing_period)
                >> map(fix_world_trade)
                )
prepped_lines[:20]

["Gordon M. Aamoth, Jr., 32, Sandler O'Neill + Partners, World Trade Center.",
 'Edelmiro Abad, 54, Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.',
 'Marie Rose Abad, 49, Keefe, Bruyette&Woods, Inc., World Trade Center.',
 'Andrew Anthony Abate, 37, Melville, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Vincent Paul Abate, 40, Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.',
 'Laurence Christopher Abel, 37, New York City, Cantor Fitzgerald, World Trade Center.',
 'Alona Abraham, 30, Ashdod, Israel, Passenger, United 175, World Trade Center.',
 'William F. Abrahamson, 55, Westchester County, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.',
 'Richard Anthony Aceto, 42, Marsh&McLennan Companies, Inc., World Trade Center.',
 'Heinrich Bernhard Ackermann, 38, Aon Corporation, World Trade Center.',
 'Paul Acquaviva, 29, Glen Rock, N.J., Cantor Fitzgerald, World Trade Center.',
 'Christian Adams, 37, Passenger, United 93, Shanksville, Pa.',


## Regular expression from lab 2

Below I have attempted to combine all of the regular expressions from lab 2

In [8]:
import re
line_parts = re.compile('^(.+), (\?\?|\d{1,3}),(.*?)( Passenger,| Flight Crew,)?( United \d{2,3},| American \d{2,3},)?( World Trade Center| Pentagon| Shanksville, Pa)(, died \d{1,2}/\d{1,2}/\d{1,2})?\.$')

In [9]:
prepped_lines[2402]

'Jesus Sanchez, 45, Flight Crew, United 175, World Trade Center.'

In [10]:
line_parts.search(prepped_lines[2402]).groups()

('Jesus Sanchez',
 '45',
 '',
 ' Flight Crew,',
 ' United 175,',
 ' World Trade Center',
 None)

#### Always check for non-matches

In [11]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

## Verbose regular expressions

**Pros:**
* Spread over multiple lines
* Allow comments

**Cons:**
* Ignore white space outside `()`
* Require escaping spaces `\ `

In [11]:
# Without Using VERBOSE 
regex_email = re.compile(r'^([a-z0-9_\.-]+)@([0-9a-z\.-]+)\.([a-z\.]{2, 6})$')

In [12]:
# Using VERBOSE 
regex_email = re.compile(r""" 
                        ^([a-z0-9_\.-]+)			 # local Part 
                        @							 # single @ sign 
                        ([0-9a-z\.-]+)			 	 # Domain name 
                        \.						 	 # single Dot . 
                        ([a-z]{2,6})$				 # Top level Domain 
                        """,re.VERBOSE)

## Another example.

This example, from the Python docs, shows how to space out an OR section across multiple lines.

In [13]:
charref = re.compile(r"""
 &[#]                # Start of a numeric entity reference
 (
     0[0-7]+         # Octal form
   | [0-9]+          # Decimal form
   | x[0-9a-fA-F]+   # Hexadecimal form
 )
 ;                   # Trailing semicolon
""", re.VERBOSE)

## Cleaning up our regular expr

<h2> <font color="red"> Exercise 4.6.1 - Clean up the regular expression </font> </h2>

To clean up the regular expression, 

1. Replace all spaces with `\ ` or `\s` (I prefer the second)
2. Turn the string into a multi-line string.
3. Spread the parts over many lines
4. Add comments.

In the following strucutre, spaces are reconginzed as formatting, not as actualt regex characters. At a minimun, the spaces need to be replaces with at least \ 

In [15]:
line_parts2 = re.compile(r"""

^(.+),\s        #front part
    (
    \?\?|\d{1,3}),(.*?)(\sPassenger,|   #may be a passenger
    \sFlight Crew,                      #or flight crew
    )?                                  #or neither

    (
        \sUnited\s\d{2,3},|             #may have been on an american flight
        \sAmerican\s\d{2,3},            #a United Flight
    )?                                  #or neither
    ( 
        \sWorld\sTrade\sCenter|         #could have died at WTC
        \sPentagon|                     #Pentagon
        \sShanksville,\sPa              #or PN
    )
    (
        ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}#grab trailing details if they exist
    )?
    \.$
     
    
    
""",re.VERBOSE
)

In [17]:
line_parts2.search(prepped_lines[2402]).groups() #looks good

('Jesus Sanchez',
 '45',
 ' Flight Crew,',
 None,
 ' United 175,',
 ' World Trade Center',
 None)

In [16]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts2.search(l)] #catches everything

[]

## Progress so far

In [18]:
# Imports
from composable import pipeable
from composable.strict import map

In [19]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [20]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
# New
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))

In [18]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [23]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(get_line_parts)
                )
split_lines[0:5]

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  '')]

## Pulling out and cleaning up names

Sometimes it is useful to pull the various columns apart and clean them up separately.  To illustrate, will will pull out and clean up the names. We can do this using the `get` function from `toolz.curried` which *gets* the value from a list at a given index.

In [24]:
from toolz.curried import get

In [25]:
(split_lines
>> map(get(0))
)[0:10]

['Gordon M. Aamoth, Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann']

Now we can clean up these name by removing commas.

In [26]:
remove_commas = lambda s: s.replace(',', '')

(split_lines
>> map(get(0))
>> map(remove_commas)
)[0:30]

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern']

## Pulling out and cleaning up ages

NExt, we will pull out and clean the ages.  In this case, we should replace the missing values, currently `'??'`, to blanks.

In [27]:
remove_quest_mark = lambda s: s.replace('??', '')

(split_lines
>> map(get(1))
>> map(remove_quest_mark)
)[0:30]

[' 32',
 ' 54',
 ' 49',
 ' 37',
 ' 40',
 ' 37',
 ' 30',
 ' 55',
 ' 42',
 ' 38',
 ' 29',
 ' 37',
 ' 28',
 ' 61',
 ' 25',
 ' 51',
 ' 62',
 ' 28',
 ' 22',
 ' 36',
 ' 48',
 ' 32',
 ' 37',
 ' 36',
 ' 37',
 ' 35',
 ' 46',
 ' 30',
 ' 43',
 ' 74']

## Progress so far

In [28]:
# Imports
from composable import pipeable
from composable.strict import map

In [29]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [30]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
# New
remove_commas = lambda s: s.replace(',', '')
remove_quest_mark = lambda s: s.replace('??', '')

In [27]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [31]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(get_line_parts)
                )
split_lines[0:5]

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  '')]

In [67]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names[0:30]

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern']

In [236]:
ages =  (split_lines
        >> map(get(1))
        >> map(remove_quest_mark)
        >> map(Strip_spaces) #I added this after the fact for consistancy
        )
ages[0:30]

['32',
 '54',
 '49',
 '37',
 '40',
 '37',
 '30',
 '55',
 '42',
 '38',
 '29',
 '37',
 '28',
 '61',
 '25',
 '51',
 '62',
 '28',
 '22',
 '36',
 '48',
 '32',
 '37',
 '36',
 '37',
 '35',
 '46',
 '30',
 '43',
 '74']

<h2> <font color="red"> Exercise 4.6.2 - Separating and cleaning other columns. </font> </h2>

To clean up the following columns 

1. Grab the date of death and replace the missing values with `9/11/2001`
2. Grab the locations (e.g. `World Trade Center`) and remove the comma from `'Shanksville, Pa.`
3. Grab the flights.
4. Grab the passenger status.

**Note:** Be sure to strip whitespace from all of them.

In [37]:
line_parts.search(prepped_lines[2402]).groups()

('Jesus Sanchez',
 ' 45',
 ' Flight Crew,',
 None,
 ' United 175,',
 ' World Trade Center',
 None)

In [69]:

# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))

# New
remove_commas = lambda s: s.replace(',', '')
remove_quest_mark = lambda s: s.replace('??', '')
replace_blank_death = lambda s: '9/11/2001' if s == '' else s
replace_01 = lambda s: s.replace('01', '2001')
Strip_spaces = lambda s: s.strip()



In [48]:
#grab  death day, replace blanks with 9/11/2001

deathdate =  (split_lines
        >> map(get(6))
        >> map(replace_blank_death)
        )
deathdate[20:40]




['9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 ', died 9/15/01',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001',
 '9/11/2001']

In [235]:
#Grab the death location and remove comma from Shanksville Pa

deathlocation =  (split_lines
        >> map(get(5))
        >> map(remove_commas)
        >> map(Strip_spaces)
        )
deathlocation[0:15]


['World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center',
 'Shanksville Pa',
 'World Trade Center',
 'World Trade Center',
 'World Trade Center']

In [72]:
#grabbing the flight and removing trailing and leading whitespace

flight =  (split_lines
        >> map(get(4))
        >> map(Strip_spaces)
        )
flight[0:20]


['',
 '',
 '',
 '',
 '',
 '',
 'United 175,',
 '',
 '',
 '',
 '',
 'United 93,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [83]:
#grab passenger status and strip spaces


passstatus =  (split_lines
        >> map(get(3))
        >> map(Strip_spaces)
        )
passstatus[0:20] 


['',
 '',
 '',
 '',
 '',
 '',
 'Passenger,',
 '',
 '',
 '',
 '',
 'Passenger,',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

## Grabbing the troubling bit

We have made significant progress, but still need to work on the third entry, which contains the hometown and employment information.  Again, we can do this using the `get` function from `toolz.curried` which *gets* the value from a list at a given index.

In [33]:
troubling_bit = (split_lines
                >> map(get(2))
                )
troubling_bit[:20]

[" Sandler O'Neill + Partners,",
 ' Brooklyn, N.Y., Fiduciary Trust Company International,',
 ' Keefe, Bruyette&Woods, Inc.,',
 ' Melville, N.Y., Cantor Fitzgerald,',
 ' Brooklyn, N.Y., Cantor Fitzgerald,',
 ' New York City, Cantor Fitzgerald,',
 ' Ashdod, Israel,',
 ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Aon Corporation,',
 ' Glen Rock, N.J., Cantor Fitzgerald,',
 '',
 ' Cantor Fitzgerald,',
 ' Fuji Bank, Ltd. security,',
 ' Cantor Fitzgerald,',
 ' New York City, Windows on the World,',
 ' Bronx, N.Y., New York Metropolitan Transportation Council,',
 ' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
 ' New York City, Fred Alger Management, Inc.,',
 ' Bronx, N.Y., Windows on the World,']

## Progressively filtering out states

We will start by matching two of the most common states, NY and NJ.

In [34]:
state = re.compile(', (N\.Y\.|N\.J\.),?')
# Rows that match
[(l, state.search(l)) for l in troubling_bit][:20]

[(" Sandler O'Neill + Partners,", None),
 (' Brooklyn, N.Y., Fiduciary Trust Company International,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Keefe, Bruyette&Woods, Inc.,', None),
 (' Melville, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Brooklyn, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' New York City, Cantor Fitzgerald,', None),
 (' Ashdod, Israel,', None),
 (' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(19, 26), match=', N.Y.,'>),
 (' Marsh&McLennan Companies, Inc.,', None),
 (' Aon Corporation,', None),
 (' Glen Rock, N.J., Cantor Fitzgerald,',
  <re.Match object; span=(10, 17), match=', N.J.,'>),
 ('', None),
 (' Cantor Fitzgerald,', None),
 (' Fuji Bank, Ltd. security,', None),
 (' Cantor Fitzgerald,', None),
 (' New York City, Windows on the World,', None),
 (' Bronx, N.Y., New York Metropolitan Transportation Council,',
  <re.Match object;

and inspecting all rows that don't match for additional states or problems

In [35]:
[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)][:20]

[(0, " Sandler O'Neill + Partners,"),
 (2, ' Keefe, Bruyette&Woods, Inc.,'),
 (5, ' New York City, Cantor Fitzgerald,'),
 (6, ' Ashdod, Israel,'),
 (8, ' Marsh&McLennan Companies, Inc.,'),
 (9, ' Aon Corporation,'),
 (11, ''),
 (12, ' Cantor Fitzgerald,'),
 (13, ' Fuji Bank, Ltd. security,'),
 (14, ' Cantor Fitzgerald,'),
 (15, ' New York City, Windows on the World,'),
 (18, ' New York City, Fred Alger Management, Inc.,'),
 (20, ' Cantor Fitzgerald,'),
 (22, ' Windows on the World,'),
 (23, ' Cantor Fitzgerald,'),
 (24, ' Fiduciary Trust Company International,'),
 (29, ' New Jersey, New York State Department of Taxation and Finance,'),
 (32, ' Marsh&McLennan consultant,'),
 (33, ' Summit Security Services, Inc.,'),
 (34, ' New York City, Marsh&McLennan Companies, Inc.,')]

## Fixing a common problem.

Notice that many rows simply contain ` New York City,` without the state.  Let's fix this problem in our preprocessing step.

In [36]:
grouped_lines[41]

'David D. Alger, 57, New York City, Fred Alger Management, Inc., World Trade Center.'

In [36]:
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))
grouped_lines[41] >> fix_nyc

'David D. Alger, 57, New York City, N.Y., Fred Alger Management, Inc., World Trade Center.'

## Progress so far

In [89]:
# Imports
from composable import pipeable
from composable.strict import map

In [90]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [91]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
remove_commas = lambda s: s.replace(',', '')
# New
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))

In [92]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [87]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                )
split_lines[:10]

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Laurence Christopher Abel',
  ' 37',
  ' New York City, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Alona Abraham',
  ' 30',
  ' Ashdod, Israel,',
  ' Passenger,',
  ' United 175,',
  ' World Trade Center',
  ''),
 ('William F. Abrahamson',
  ' 55',
  ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Richard Anth

In [88]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names[:10]

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann']

In [156]:
troubling_bit = (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                >> map(get(2))
                )
troubling_bit[:20]

[" Sandler O'Neill + Partners,",
 ' Brooklyn, N.Y., Fiduciary Trust Company International,',
 ' Keefe, Bruyette&Woods, Inc.,',
 ' Melville, N.Y., Cantor Fitzgerald,',
 ' Brooklyn, N.Y., Cantor Fitzgerald,',
 ' New York City, N.Y., Cantor Fitzgerald,',
 ' Ashdod, Israel,',
 ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Aon Corporation,',
 ' Glen Rock, N.J., Cantor Fitzgerald,',
 '',
 ' Cantor Fitzgerald,',
 ' Fuji Bank, Ltd. security,',
 ' Cantor Fitzgerald,',
 ' New York City, N.Y., Windows on the World,',
 ' Bronx, N.Y., New York Metropolitan Transportation Council,',
 ' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
 ' New York City, N.Y., Fred Alger Management, Inc.,',
 ' Bronx, N.Y., Windows on the World,']

## Adding more states

Next, we will start adding start to our pattern, and again looking for additional states/problems.  For example, let's add the `Mass.` and `D.C.` patterns.

In [82]:
state = re.compile(', (N\.Y\.|N\.J\.|Mass\.|D\.C\.),?')
[(l, state.search(l)) for l in troubling_bit if state.search(l)][0:30]

[(' Brooklyn, N.Y., Fiduciary Trust Company International,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Melville, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' Brooklyn, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(9, 16), match=', N.Y.,'>),
 (' New York City, N.Y., Cantor Fitzgerald,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(19, 26), match=', N.Y.,'>),
 (' Glen Rock, N.J., Cantor Fitzgerald,',
  <re.Match object; span=(10, 17), match=', N.J.,'>),
 (' New York City, N.Y., Windows on the World,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' Bronx, N.Y., New York Metropolitan Transportation Council,',
  <re.Match object; span=(6, 13), match=', N.Y.,'>),
 (' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
  <re.Match object; span=(14, 21), match=', N.Y.,'>),
 (' New York City, N.Y., Fred Alger Management, In

In [81]:
[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)]

[(0, " Sandler O'Neill + Partners,"),
 (2, ' Keefe, Bruyette&Woods, Inc.,'),
 (6, ' Ashdod, Israel,'),
 (8, ' Marsh&McLennan Companies, Inc.,'),
 (9, ' Aon Corporation,'),
 (11, ''),
 (12, ' Cantor Fitzgerald,'),
 (13, ' Fuji Bank, Ltd. security,'),
 (14, ' Cantor Fitzgerald,'),
 (20, ' Cantor Fitzgerald,'),
 (22, ' Windows on the World,'),
 (23, ' Cantor Fitzgerald,'),
 (24, ' Fiduciary Trust Company International,'),
 (32, ' Marsh&McLennan consultant,'),
 (33, ' Summit Security Services, Inc.,'),
 (43, ' Cantor Fitzgerald,'),
 (45, ' Cantor Fitzgerald,'),
 (51, ' Cantor Fitzgerald, Forte Food Service,'),
 (52, ' Windows on the World,'),
 (54, ' Windows on the World,'),
 (55, ' Marsh&McLennan Companies, Inc.,'),
 (56, ' Fiduciary Trust Company International,'),
 (57, ' ABM Industries Inc.,'),
 (58, ' New York City Fire Department,'),
 (59, ' Port Authority of New York and New Jersey first responders,'),
 (61, ' Port Authority Police Department,'),
 (69, ' Cantor Fitzgerald,'),
 (71, '

<h2> <font color="red"> Exercise 4.6.2 - Continue the process. </font> </h2>

Now it is your turn.  You should

1. Keep adding states to the pattern.
2. Add preprocessing steps to fix any issues.

In [99]:
state = re.compile(',\s(N\.Y\.|N\.J\.|New\sJersey|Mass\.|D\.C\.|DC|Ill\.|N\.M\.|NM|New Mexico|Va|Va\.|Virginia),?')


[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)][0:50]

[(0, " Sandler O'Neill + Partners,"),
 (2, ' Keefe, Bruyette&Woods, Inc.,'),
 (6, ' Ashdod, Israel,'),
 (8, ' Marsh&McLennan Companies, Inc.,'),
 (9, ' Aon Corporation,'),
 (11, ''),
 (12, ' Cantor Fitzgerald,'),
 (13, ' Fuji Bank, Ltd. security,'),
 (14, ' Cantor Fitzgerald,'),
 (20, ' Cantor Fitzgerald,'),
 (22, ' Windows on the World,'),
 (23, ' Cantor Fitzgerald,'),
 (24, ' Fiduciary Trust Company International,'),
 (29, ' New Jersey, New York State Department of Taxation and Finance,'),
 (32, ' Marsh&McLennan consultant,'),
 (33, ' Summit Security Services, Inc.,'),
 (43, ' Cantor Fitzgerald,'),
 (45, ' Cantor Fitzgerald,'),
 (51, ' Cantor Fitzgerald, Forte Food Service,'),
 (52, ' Windows on the World,'),
 (54, ' Windows on the World,'),
 (55, ' Marsh&McLennan Companies, Inc.,'),
 (56, ' Fiduciary Trust Company International,'),
 (57, ' ABM Industries Inc.,'),
 (58, ' New York City Fire Department,'),
 (59, ' Port Authority of New York and New Jersey first responders,'),
 (61, ' 

In [157]:
#if I saw a State, i did all variations I could think of for that state

state = re.compile(',?\s(N\.Y\.|New\sYork|NY|N\.J\.|New\sJersey|NJ|Mass\.|D\.C\.|DC|Ill\.|N\.M\.|NM|New Mexico|Va|V\.A\.|Va\.|Virginia|NC|N\.C\.|North Carolina|Md|Maryland|M\.D\.|Mary\.|G\.A\.|GA|Georgia|Atlanta,\sGA|I\.A\.|IA|Iowa|Mass|Mass.|MA|M\.A\.|California|Calif\.|CA|C\.A\.|Arizona|Ariz\.|AZ|A\.Z\.|Maine|ME|M\.E\.|Ohio|OH|O\.H\.|Rhode Island|R\.I\.|RI|Florida|Fla\.|FL|F\.L\.|Ill\.|Illinois|I\.A\.|New\sHampshire|NH|N\.H\.|Pennsylvania|Pa\.|PA|P\.A\.|Hawaii|HI|H\.I\.|Louisiana|LA|L\.A\.|La\.|Texas|TX|T\.X\.|Connecticut|CA|C\.A\.|Conn\.|Utah|UT|U\.T\.|Germany|Israel|Manitoba,\sCanada|Canada|New\sSouth\sWales,\sAustralia|Australia|U\.K\.|England|England,\sUnited\sKingdom|India|Japan),?')


len([(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)]) 

#from here, it appears that the rest of the answers are missing this field, this will be dealth with up ahead
#when we split the data


645

In [158]:
[(i, l) for i, l in enumerate(troubling_bit) if not state.search(l)][0:10] 

[(0, " Sandler O'Neill + Partners,"),
 (2, ' Keefe, Bruyette&Woods, Inc.,'),
 (8, ' Marsh&McLennan Companies, Inc.,'),
 (9, ' Aon Corporation,'),
 (11, ''),
 (12, ' Cantor Fitzgerald,'),
 (13, ' Fuji Bank, Ltd. security,'),
 (14, ' Cantor Fitzgerald,'),
 (20, ' Cantor Fitzgerald,'),
 (22, ' Windows on the World,')]

<h2> <font color="red"> Exercise 4.6.3 - Make your solution verbose </font> </h2>

Now make your solution to the last problem verbose.  Also reorder the cases so that similar cases are close and add comments.  Finally, change the regular expression to capture the parts before and after the state.

In [162]:

#in the future, i'll refrain from adding in additional details I assume may get hits based on what
#already did, as the following below is likely way more complicated then it needs to be

state = re.compile('''
^(.*?),?\s #grab the front bit and the possible comma

(                   #variations of the state, with all needed ones a possibly a few extra
    N\.Y\.|         #NY 
    New\sYork|
    NY|
    N\.J\.|
    
    New\sJersey|    #NJ
    NJ|

    D\.C\.|         #DC
    DC|

    N\.M\.|         #NM
    NM|
    New Mexico|
    
    Va|             #VA
    V\.A\.|
    Va\.|
    Virginia|

    NC|             #NC
    N\.C\.|
    North Carolina|
    
    Md|             #MD
    Maryland|
    M\.D\.|
    Mary\.|
    MD|

    G\.A\.|         #GA
    GA|
    Georgia|
    Atlanta,\sGA|
    
    I\.A\.|         #IA
    IA|
    Iowa|
    
    Mass|           #MA, here odd patterns like Mass. show up fairly often
    Mass.|
    MA|
    M\.A\.|
    
    California|     #CA
    Calif\.|
    CA|
    C\.A\.|
    
    Arizona|        #AZ
    Ariz\.|
    AZ|
    A\.Z\.|
    
    Maine|          #ME
    ME|
    M\.E\.|
    
    Ohio|           O#H
    OH|
    O\.H\.|
    
    Rhode Island|   #RI
    R\.I\.|
    RI|
    
    Florida|        #FL
    Fla\.|
    FL|
    F\.L\.|
    
    Ill\.|          #IL
    Illinois|
    I\.A\.|

    New\sHampshire| #NH 
    NH|
    N\.H\.|

    Pennsylvania|   #PA
    Pa\.|
    PA|
    P\.A\.|
    
    Hawaii|         #HI
    HI|
    H\.I\.|
    
    Louisiana|      #LA
    LA|
    L\.A\.|
    La\.|
    
    Texas|          #TX
    TX|
    T\.X\.|
    
    Connecticut|    #CA
    CA|
    C\.A\.|
    Conn\.|
    
    Utah|           #UT
    UT|
    U\.T\.|
    
                    #these are misc. including other countries and odd variations
    Germany|
    Israel|
    Manitoba,\sCanada|
    Canada|
    New\sSouth\sWales,\sAustralia|
    Australia|
    U\.K\.|
    England|
    England,\sUnited\sKingdom|
    India|
    Japan
    
)



,
(.*?)$
''', re.VERBOSE)

[(i, l) for i, l in enumerate(troubling_bit) if state.search(l)][0:50] #works whenever there is a state


[(1, ' Brooklyn, N.Y., Fiduciary Trust Company International,'),
 (3, ' Melville, N.Y., Cantor Fitzgerald,'),
 (4, ' Brooklyn, N.Y., Cantor Fitzgerald,'),
 (5, ' New York City, N.Y., Cantor Fitzgerald,'),
 (6, ' Ashdod, Israel,'),
 (7, ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,'),
 (10, ' Glen Rock, N.J., Cantor Fitzgerald,'),
 (15, ' New York City, N.Y., Windows on the World,'),
 (16, ' Bronx, N.Y., New York Metropolitan Transportation Council,'),
 (17, ' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,'),
 (18, ' New York City, N.Y., Fred Alger Management, Inc.,'),
 (19, ' Bronx, N.Y., Windows on the World,'),
 (21, ' Manalapan, N.J., Cantor Fitzgerald,'),
 (25, ' Belle Harbor, N.Y., New York City Fire Department,'),
 (26, ' Port Washington, N.Y., Cantor Fitzgerald,'),
 (27, ' Hoboken, N.J., Keefe, Bruyette&Woods, Inc.,'),
 (28, ' Huntington, N.Y., New York City Fire Department,'),
 (29, ' New Jersey, New York State Department of Taxation and Finance,'),
 (30, ' 

## Splitting the troubling bit

Now that we have a way to identify rows that have home addresses (through matching the state), we will split up this data.  We will do this by considering three cases.

1. Blank entry become three blanks (for town, state, employer).
2. Lines that match the states regex will get split by this pattern.
3. The remaining lines hold only the employer and become `'','',entry`

In [163]:
def split_troubling_bit(entry):
    if len(entry) == 0:
        return ('', '', '')
    elif state.search(entry):
        return state.search(entry).groups(default='')
    else:
        return ('', '', entry)

In [164]:
( troubling_bit
 >> map(split_troubling_bit)
)[0:30]

[('', '', " Sandler O'Neill + Partners,"),
 (' Brooklyn', 'N.Y.', ' Fiduciary Trust Company International,'),
 ('', '', ' Keefe, Bruyette&Woods, Inc.,'),
 (' Melville', 'N.Y.', ' Cantor Fitzgerald,'),
 (' Brooklyn', 'N.Y.', ' Cantor Fitzgerald,'),
 (' New York City', 'N.Y.', ' Cantor Fitzgerald,'),
 (' Ashdod', 'Israel', ''),
 (' Westchester County', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Aon Corporation,'),
 (' Glen Rock', 'N.J.', ' Cantor Fitzgerald,'),
 ('', '', ''),
 ('', '', ' Cantor Fitzgerald,'),
 ('', '', ' Fuji Bank, Ltd. security,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' New York City', 'N.Y.', ' Windows on the World,'),
 (' Bronx', 'N.Y.', ' New York Metropolitan Transportation Council,'),
 (' New Hyde Park', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 (' New York City', 'N.Y.', ' Fred Alger Management, Inc.,'),
 (' Bronx', 'N.Y.', ' Windows on the World,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' Manalapan', '

## Progress so far

In [165]:
# Imports
from composable import pipeable
from composable.strict import map

In [166]:
# Reg Ex for a line
line_parts = re.compile(r'''^(.+),
(
      \s\?\?                          # ??
    | \s\d{1,3}                       # or age
),
(.*?)                                 # Includes hometown and 
(
        \sPassenger,                  # Optional flight status
    |   \sFlightsCrew,
)?
(
      \sUnited\s\d{2,3},              # Optional flight
    | \sAmericans\d{2,3},
)?
(
       \sWorld\sTrade\sCenter         # Location
    |  \sPentagon
    |  \sShanksville,\sPa
)
(
    ,\sdied\s\d{1,2}/\d{1,2}/\d{1,2}  # Optional date of death
)?
\.$''', re.VERBOSE)

In [167]:
# Helper functions
add_missing_period = pipeable(lambda line: line if line.endswith('.') else line + '.' )
fix_world_trade = pipeable(lambda line: line.replace('WorldTrade', 'World Trade'))
get_line_parts = pipeable(lambda line: line_parts.search(line).groups(default=''))
remove_commas = lambda s: s.replace(',', '')
# New
fix_nyc = pipeable(lambda line: line.replace(', New York City,', ', New York City, N.Y.,'))

In [53]:
[(i, l) for i, l in enumerate(prepped_lines) if not line_parts.search(l)]

[]

In [168]:
split_lines =  (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                )
split_lines[0:10]

[('Gordon M. Aamoth, Jr.',
  ' 32',
  " Sandler O'Neill + Partners,",
  '',
  '',
  ' World Trade Center',
  ''),
 ('Edelmiro Abad',
  ' 54',
  ' Brooklyn, N.Y., Fiduciary Trust Company International,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Marie Rose Abad',
  ' 49',
  ' Keefe, Bruyette&Woods, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Andrew Anthony Abate',
  ' 37',
  ' Melville, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Vincent Paul Abate',
  ' 40',
  ' Brooklyn, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Laurence Christopher Abel',
  ' 37',
  ' New York City, N.Y., Cantor Fitzgerald,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Alona Abraham',
  ' 30',
  ' Ashdod, Israel,',
  ' Passenger,',
  ' United 175,',
  ' World Trade Center',
  ''),
 ('William F. Abrahamson',
  ' 55',
  ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
  '',
  '',
  ' World Trade Center',
  ''),
 ('Richard Anth

In [169]:
names =  (split_lines
        >> map(get(0))
        >> map(remove_commas)
        )
names[0:30]

['Gordon M. Aamoth Jr.',
 'Edelmiro Abad',
 'Marie Rose Abad',
 'Andrew Anthony Abate',
 'Vincent Paul Abate',
 'Laurence Christopher Abel',
 'Alona Abraham',
 'William F. Abrahamson',
 'Richard Anthony Aceto',
 'Heinrich Bernhard Ackermann',
 'Paul Acquaviva',
 'Christian Adams',
 'Donald LaRoy Adams',
 'Patrick Adams',
 'Shannon Lewis Adams',
 'Stephen George Adams',
 'Ignatius Udo Adanga',
 'Christy A. Addamo',
 'Terence Edward Adderley Jr.',
 'Sophia B. Addo',
 'Lee Adler',
 'Daniel Thomas Afflitto',
 'Emmanuel Akwasi Afuakwah',
 'Alok Agarwal',
 'Mukul Kumar Agarwala',
 'Joseph Agnello',
 'David Scott Agnes',
 'Joao Alberto da Fonseca Aguiar Jr.',
 'Brian G. Ahearn',
 'Jeremiah Joseph Ahern']

In [80]:
troubling_bit = (grouped_lines
                >> map(add_missing_period)
                >> map(fix_world_trade)
                >> map(fix_nyc)
                >> map(get_line_parts)
                >> map(get(2))
                )
troubling_bit[0:30]

[" Sandler O'Neill + Partners,",
 ' Brooklyn, N.Y., Fiduciary Trust Company International,',
 ' Keefe, Bruyette&Woods, Inc.,',
 ' Melville, N.Y., Cantor Fitzgerald,',
 ' Brooklyn, N.Y., Cantor Fitzgerald,',
 ' New York City, N.Y., Cantor Fitzgerald,',
 ' Ashdod, Israel,',
 ' Westchester County, N.Y., Marsh&McLennan Companies, Inc.,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Aon Corporation,',
 ' Glen Rock, N.J., Cantor Fitzgerald,',
 '',
 ' Cantor Fitzgerald,',
 ' Fuji Bank, Ltd. security,',
 ' Cantor Fitzgerald,',
 ' New York City, N.Y., Windows on the World,',
 ' Bronx, N.Y., New York Metropolitan Transportation Council,',
 ' New Hyde Park, N.Y., Marsh&McLennan Companies, Inc.,',
 ' New York City, N.Y., Fred Alger Management, Inc.,',
 ' Bronx, N.Y., Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Manalapan, N.J., Cantor Fitzgerald,',
 ' Windows on the World,',
 ' Cantor Fitzgerald,',
 ' Fiduciary Trust Company International,',
 ' Belle Harbor, N.Y., New York City Fire Department,',

In [170]:
state = re.compile('''
^(.*?)
,?\s                    # Optional comman
(
       N\.Y\.           
    |  N\.J\.
    |  D\.C\.
    |  N\.H\.
    |  N\.M\.
    |  N\.C\.
    |  R.I.
    |  Md\.
    |  Pa\.
    |  Va\.
    |  Ga\.
    |  La\.
    |  Mass\.
    |  Calif\.
    |  Ariz\.
    |  Fla\.
    |  Ill\.
    |  Conn\.
    |  Hawaii
    |  Iowa
    |  Maine
    |  New\sHampshire
    |  New\sJersey
    |  New\sYork
    |  Ohio
    |  Pennsylvania
    |  Texas
    |  Utah
    |  Virginia
    |  Japan
    |  India
    |  Germany
    |  Manitoba,\sCanada
    |  New\sSouth\sWales,\sAustralia
    |  England,\sUnited\sKingdom
)
,
(.*?)$
''', re.VERBOSE)

In [183]:
Splittb =( troubling_bit
            >> map(split_troubling_bit)
        )



( troubling_bit
 >> map(split_troubling_bit)
)[0:30]

[('', '', " Sandler O'Neill + Partners,"),
 (' Brooklyn', 'N.Y.', ' Fiduciary Trust Company International,'),
 ('', '', ' Keefe, Bruyette&Woods, Inc.,'),
 (' Melville', 'N.Y.', ' Cantor Fitzgerald,'),
 (' Brooklyn', 'N.Y.', ' Cantor Fitzgerald,'),
 (' New York City', 'N.Y.', ' Cantor Fitzgerald,'),
 ('', '', ' Ashdod, Israel,'),
 (' Westchester County', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Marsh&McLennan Companies, Inc.,'),
 ('', '', ' Aon Corporation,'),
 (' Glen Rock', 'N.J.', ' Cantor Fitzgerald,'),
 ('', '', ''),
 ('', '', ' Cantor Fitzgerald,'),
 ('', '', ' Fuji Bank, Ltd. security,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' New York City', 'N.Y.', ' Windows on the World,'),
 (' Bronx', 'N.Y.', ' New York Metropolitan Transportation Council,'),
 (' New Hyde Park', 'N.Y.', ' Marsh&McLennan Companies, Inc.,'),
 (' New York City', 'N.Y.', ' Fred Alger Management, Inc.,'),
 (' Bronx', 'N.Y.', ' Windows on the World,'),
 ('', '', ' Cantor Fitzgerald,'),
 (' Manalapan'

<h2> <font color="red"> Exercise 4.5.4 </font> </h2>

Clean up each part of the troubling bits, then comma join this section into 1 string.

**Hint:** Be sure to remove any problematic commas.

In [175]:
#moved down here for convienance

remove_commas = lambda s: s.replace(',', '')
remove_periods = lambda s: s.replace('.', '')
remove_quest_mark = lambda s: s.replace('??', '')
replace_blank_death = lambda s: '9/11/2001' if s == '' else s
replace_01 = lambda s: s.replace('01', '2001')
Strip_spaces = lambda s: s.strip()

In [184]:
#starting with state

stateofdeath =  (Splittb
        >> map(get(1))
        )
stateofdeath[0:15]

['',
 'N.Y.',
 '',
 'N.Y.',
 'N.Y.',
 'N.Y.',
 '',
 'N.Y.',
 '',
 '',
 'N.J.',
 '',
 '',
 '',
 '']

In [189]:
#remove periods, commas, and remove extra whitespace 

stateofdeath =  (Splittb
        >> map(get(1))
        >> map(remove_periods)
        >> map(remove_commas)
        >> map(Strip_spaces)
        )
stateofdeath[:50] 

#stateofdeath 

#looks good, state is ready. Some categories still need evaluations (like Isreal and Japan)
#and one may wish to drop the whole data set into openrefine after this is all done to quickly
#collapse categories


['',
 'NY',
 '',
 'NY',
 'NY',
 'NY',
 '',
 'NY',
 '',
 '',
 'NJ',
 '',
 '',
 '',
 '',
 'NY',
 'NY',
 'NY',
 'NY',
 'NY',
 '',
 'NJ',
 '',
 '',
 '',
 'NY',
 'NY',
 'NJ',
 'NY',
 'New Jersey',
 'NY',
 'NY',
 '',
 '',
 'NY',
 'NJ',
 'NY',
 'NJ',
 'NY',
 'NY',
 'NY',
 'NY',
 'NY',
 '',
 'NY',
 '',
 'NY',
 'NY',
 'NJ',
 'Mass']

In [191]:
#Next up is city, which will follow a similar structure

cityofdeath =  (Splittb
        >> map(get(0))
        )
cityofdeath[0:15]


['',
 'Brooklyn',
 '',
 'Melville',
 'Brooklyn',
 'New York City',
 '',
 'Westchester County',
 '',
 '',
 'Glen Rock',
 '',
 '',
 '',
 '',
 'New York City',
 'Bronx',
 'New Hyde Park',
 'New York City',
 'Bronx',
 '',
 'Manalapan',
 '',
 '',
 '',
 'Belle Harbor',
 'Port Washington',
 'Hoboken',
 'Huntington',
 '',
 'Forest Hills',
 'Brooklyn',
 '',
 '',
 'New York City',
 'Westfield',
 'Staten Island',
 'Emerson',
 'Upper Nyack',
 'New York City',
 'Staten Island',
 'New York City',
 'Brooklyn',
 '',
 'Brooklyn',
 '',
 'Rockaway Beach',
 'Brooklyn',
 'River Edge',
 'Stoneham']

In [192]:
#cleaning it will involve all of the same tools


cityofdeath =  (Splittb
        >> map(get(0))
        >> map(remove_periods)
        >> map(remove_commas) 
        >> map(Strip_spaces)
        )
cityofdeath[0:50] #looks good, but one still should use openrefine to check it after were done

['',
 'Brooklyn',
 '',
 'Melville',
 'Brooklyn',
 'New York City',
 '',
 'Westchester County',
 '',
 '',
 'Glen Rock',
 '',
 '',
 '',
 '',
 'New York City',
 'Bronx',
 'New Hyde Park',
 'New York City',
 'Bronx',
 '',
 'Manalapan',
 '',
 '',
 '',
 'Belle Harbor',
 'Port Washington',
 'Hoboken',
 'Huntington',
 '',
 'Forest Hills',
 'Brooklyn',
 '',
 '',
 'New York City',
 'Westfield',
 'Staten Island',
 'Emerson',
 'Upper Nyack',
 'New York City',
 'Staten Island',
 'New York City',
 'Brooklyn',
 '',
 'Brooklyn',
 '',
 'Rockaway Beach',
 'Brooklyn',
 'River Edge',
 'Stoneham']

In [193]:
#Finally, we have company at the end


company =  (Splittb
        >> map(get(2))
        )
company[0:15]


[" Sandler O'Neill + Partners,",
 ' Fiduciary Trust Company International,',
 ' Keefe, Bruyette&Woods, Inc.,',
 ' Cantor Fitzgerald,',
 ' Cantor Fitzgerald,',
 ' Cantor Fitzgerald,',
 ' Ashdod, Israel,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Marsh&McLennan Companies, Inc.,',
 ' Aon Corporation,',
 ' Cantor Fitzgerald,',
 '',
 ' Cantor Fitzgerald,',
 ' Fuji Bank, Ltd. security,',
 ' Cantor Fitzgerald,']

In [194]:
#same applications as before

company =  (Splittb
        >> map(get(2))
        >> map(remove_periods)
        >> map(remove_commas) 
        >> map(Strip_spaces)
        )
company[0:50] #same caveats apply as before


["Sandler O'Neill + Partners",
 'Fiduciary Trust Company International',
 'Keefe Bruyette&Woods Inc',
 'Cantor Fitzgerald',
 'Cantor Fitzgerald',
 'Cantor Fitzgerald',
 'Ashdod Israel',
 'Marsh&McLennan Companies Inc',
 'Marsh&McLennan Companies Inc',
 'Aon Corporation',
 'Cantor Fitzgerald',
 '',
 'Cantor Fitzgerald',
 'Fuji Bank Ltd security',
 'Cantor Fitzgerald',
 'Windows on the World',
 'New York Metropolitan Transportation Council',
 'Marsh&McLennan Companies Inc',
 'Fred Alger Management Inc',
 'Windows on the World',
 'Cantor Fitzgerald',
 'Cantor Fitzgerald',
 'Windows on the World',
 'Cantor Fitzgerald',
 'Fiduciary Trust Company International',
 'New York City Fire Department',
 'Cantor Fitzgerald',
 'Keefe Bruyette&Woods Inc',
 'New York City Fire Department',
 'New York State Department of Taxation and Finance',
 'Cantor Fitzgerald',
 'Windows on the World',
 'Marsh&McLennan consultant',
 'Summit Security Services Inc',
 'Marsh&McLennan Companies Inc',
 'Cantor Fitzgerald

In [200]:
#rejoin all three back together

Fixed_tbit = list(zip(cityofdeath,stateofdeath,company)) #zip gets what we want easily

Fixed_tbit[:30]

[('', '', "Sandler O'Neill + Partners"),
 ('Brooklyn', 'NY', 'Fiduciary Trust Company International'),
 ('', '', 'Keefe Bruyette&Woods Inc'),
 ('Melville', 'NY', 'Cantor Fitzgerald'),
 ('Brooklyn', 'NY', 'Cantor Fitzgerald'),
 ('New York City', 'NY', 'Cantor Fitzgerald'),
 ('', '', 'Ashdod Israel'),
 ('Westchester County', 'NY', 'Marsh&McLennan Companies Inc'),
 ('', '', 'Marsh&McLennan Companies Inc'),
 ('', '', 'Aon Corporation'),
 ('Glen Rock', 'NJ', 'Cantor Fitzgerald'),
 ('', '', ''),
 ('', '', 'Cantor Fitzgerald'),
 ('', '', 'Fuji Bank Ltd security'),
 ('', '', 'Cantor Fitzgerald'),
 ('New York City', 'NY', 'Windows on the World'),
 ('Bronx', 'NY', 'New York Metropolitan Transportation Council'),
 ('New Hyde Park', 'NY', 'Marsh&McLennan Companies Inc'),
 ('New York City', 'NY', 'Fred Alger Management Inc'),
 ('Bronx', 'NY', 'Windows on the World'),
 ('', '', 'Cantor Fitzgerald'),
 ('Manalapan', 'NJ', 'Cantor Fitzgerald'),
 ('', '', 'Windows on the World'),
 ('', '', 'Cantor Fitzg

In [203]:
Fixed_tbit_list = [list(row) for row in zip(cityofdeath, stateofdeath, company)]

Fixed_tbit_list[:30] #if having a tuple makes an issue, can also use this list comprehension solution

[['', '', "Sandler O'Neill + Partners"],
 ['Brooklyn', 'NY', 'Fiduciary Trust Company International'],
 ['', '', 'Keefe Bruyette&Woods Inc'],
 ['Melville', 'NY', 'Cantor Fitzgerald'],
 ['Brooklyn', 'NY', 'Cantor Fitzgerald'],
 ['New York City', 'NY', 'Cantor Fitzgerald'],
 ['', '', 'Ashdod Israel'],
 ['Westchester County', 'NY', 'Marsh&McLennan Companies Inc'],
 ['', '', 'Marsh&McLennan Companies Inc'],
 ['', '', 'Aon Corporation'],
 ['Glen Rock', 'NJ', 'Cantor Fitzgerald'],
 ['', '', ''],
 ['', '', 'Cantor Fitzgerald'],
 ['', '', 'Fuji Bank Ltd security'],
 ['', '', 'Cantor Fitzgerald'],
 ['New York City', 'NY', 'Windows on the World'],
 ['Bronx', 'NY', 'New York Metropolitan Transportation Council'],
 ['New Hyde Park', 'NY', 'Marsh&McLennan Companies Inc'],
 ['New York City', 'NY', 'Fred Alger Management Inc'],
 ['Bronx', 'NY', 'Windows on the World'],
 ['', '', 'Cantor Fitzgerald'],
 ['Manalapan', 'NJ', 'Cantor Fitzgerald'],
 ['', '', 'Windows on the World'],
 ['', '', 'Cantor Fitzg

## Combining the parts back together.

We can combine the parts back together using the `zip` function.

In [204]:
fixed_troubling_bits = Fixed_tbit_list

In [206]:
from composable.strict import zipOnto
from composable.list import to_list
#(zip(names, ages, fixed_troubling_bits)
# >> to_list
# >> map(comma_join)
#)

In [228]:
#Fixed_tbit_list = [list(row) for row in zip(names,ages,cityofdeath, stateofdeath, company, passstatus, flight, deathlocation, deathdate)]

output = list(zip(names,ages,cityofdeath, stateofdeath, company, passstatus, flight, deathlocation, deathdate))

output[:5]


[('Gordon M. Aamoth Jr.',
  ' 32',
  '',
  '',
  "Sandler O'Neill + Partners",
  '',
  '',
  ' World Trade Center',
  '9/11/2001'),
 ('Edelmiro Abad',
  ' 54',
  'Brooklyn',
  'NY',
  'Fiduciary Trust Company International',
  '',
  '',
  ' World Trade Center',
  '9/11/2001'),
 ('Marie Rose Abad',
  ' 49',
  '',
  '',
  'Keefe Bruyette&Woods Inc',
  '',
  '',
  ' World Trade Center',
  '9/11/2001'),
 ('Andrew Anthony Abate',
  ' 37',
  'Melville',
  'NY',
  'Cantor Fitzgerald',
  '',
  '',
  ' World Trade Center',
  '9/11/2001'),
 ('Vincent Paul Abate',
  ' 40',
  'Brooklyn',
  'NY',
  'Cantor Fitzgerald',
  '',
  '',
  ' World Trade Center',
  '9/11/2001')]

<h2> <font color="red"> Exercise 4.5.4 </font> </h2>

Use `zip` to combine all part of the data and write the result out to a file called `911_Deaths_Fixed.csv` 

In [238]:
output = list(zip(names,ages,cityofdeath, stateofdeath, company, passstatus, flight, deathlocation, deathdate))

output[:5]

[('Gordon M. Aamoth Jr.',
  '32',
  '',
  '',
  "Sandler O'Neill + Partners",
  '',
  '',
  'World Trade Center',
  '9/11/2001'),
 ('Edelmiro Abad',
  '54',
  'Brooklyn',
  'NY',
  'Fiduciary Trust Company International',
  '',
  '',
  'World Trade Center',
  '9/11/2001'),
 ('Marie Rose Abad',
  '49',
  '',
  '',
  'Keefe Bruyette&Woods Inc',
  '',
  '',
  'World Trade Center',
  '9/11/2001'),
 ('Andrew Anthony Abate',
  '37',
  'Melville',
  'NY',
  'Cantor Fitzgerald',
  '',
  '',
  'World Trade Center',
  '9/11/2001'),
 ('Vincent Paul Abate',
  '40',
  'Brooklyn',
  'NY',
  'Cantor Fitzgerald',
  '',
  '',
  'World Trade Center',
  '9/11/2001')]

In [239]:
import csv


#ready for further cleaning
with open('911_Deaths_Fixed.csv','w') as out:
    csv_out=csv.writer(out)
    #csv_out.writerows(output) also works
    for row in output:
        csv_out.writerow(row)